In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/classicRec.png
/kaggle/input/book-recommendation-dataset/Books.csv
/kaggle/input/book-recommendation-dataset/DeepRec.png
/kaggle/input/book-recommendation-dataset/recsys_taxonomy2.png


In [2]:
import pandas as pd
import os as os


# define paths to the data files

books_path = '/kaggle/input/book-recommendation-dataset/Books.csv'
ratings_path = '/kaggle/input/book-recommendation-dataset/Ratings.csv'
users_path = '/kaggle/input/book-recommendation-dataset/Users.csv'

In [3]:
# load the datasets
books_df = pd.read_csv(books_path)
ratings_df = pd.read_csv(ratings_path)
users_df = pd.read_csv(users_path)

/tmp/ipykernel_17/25047361.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(books_path)


In [4]:
# Display some basic statistics
print("Books DataFrame Info:")
print(books_df.info(), "\n")

print("Books DataFrame Overview:")
print(books_df.head(10), "\n")
print("Books DataFrame Shape:", books_df.shape, "\n")

print("Ratings DataFrame Info:")
print(ratings_df.info(), "\n")

print("Ratings DataFrame Overview:")
print(ratings_df.head(10), "\n")
print("Ratings DataFrame Shape:", ratings_df.shape, "\n")

# Determine the number of null ratings
null_ratings_count = ratings_df['Book-Rating'].isnull().sum()
print("Number of null ratings in Ratings DataFrame:", null_ratings_count, "\n")

print("Users DataFrame Info:")
print(users_df.info(), "\n")

print("Users DataFrame Overview:")
print(users_df.head(10), "\n")
print("Users DataFrame Shape:", users_df.shape, "\n")

Books DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None 

Books DataFrame Overview:
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: Th

In [5]:
# merge ratings with books to get book details along with ratings
user_book_ratings = pd.merge(ratings_df, books_df, on="ISBN", how='inner')
explicit_ratings = user_book_ratings[user_book_ratings['Book-Rating'] != 0]

# display the first few rows of the merged dataset
print("User-book Ratings Merged Overview:")
print(explicit_ratings.head(), "\n")

# Display basic statistics of the merged dataset
print("User-book Rating Merged Info:")
print(explicit_ratings.info(), "\n")

# Check the distribution of ratings
print("Rating Distribution:")
print(explicit_ratings['Book-Rating'].value_counts(), "\n")

# Count the number of unique users and books
num_users = explicit_ratings['User-ID'].nunique()
num_books = explicit_ratings['ISBN'].nunique()
num_explicit_ratings = explicit_ratings.shape[0]
print(f"Number of unique users: {num_users}","\n")
print(f"Number of unique books: {num_books}","\n")
print(f"Number of explicit ratings: {num_explicit_ratings}","\n")

average_rating_per_book = explicit_ratings.groupby('ISBN')['Book-Rating'].mean().reset_index()
print(" ")
print("Average Rating Per Book")
print(average_rating_per_book.head(10),"\n")

print(" ")
print("Most Rated Books")
most_rated_books = explicit_ratings['ISBN'].value_counts().reset_index()
most_rated_books.columns = ['ISBN', 'count']
print(most_rated_books.head(10),"\n")


print(" ")
print("Average Rating Per user")
average_rating_per_user = explicit_ratings.groupby('User-ID')['Book-Rating'].mean().reset_index()
print(average_rating_per_user.head(10),"\n")

max_rating = explicit_ratings['Book-Rating'].max()-3
highest_rated_books = explicit_ratings[explicit_ratings['Book-Rating'] >= max_rating]

most_highly_rated_book_title = highest_rated_books['Book-Title'].value_counts().idxmax()
print("Most Highly Rated Book:", most_highly_rated_book_title)


most_highly_rated_book = highest_rated_books['ISBN'].value_counts().idxmax()
users_with_most_highly_rated_book = explicit_ratings[explicit_ratings['ISBN'] == most_highly_rated_book]['User-ID'].unique()
print("Users with Most Highly Rated Book:", len(users_with_most_highly_rated_book))

books_of_users_with_high_book = explicit_ratings[explicit_ratings['User-ID'].isin(users_with_most_highly_rated_book)]
# Display the books of users with the most highly rated book
print("Books of Users with the Most Highly Rated Book:")
print(books_of_users_with_high_book.head(), "\n")

other_books = books_of_users_with_high_book[books_of_users_with_high_book['ISBN'] != most_highly_rated_book]
# Display the other books (excluding the most highly rated book)
print("Other Books:")
print(other_books.head(), "\n")

most_common_other_book = other_books['ISBN'].value_counts().idxmax()
print("Most Common Other Book:", most_common_other_book)

users_with_both_books = books_of_users_with_high_book[
    (books_of_users_with_high_book['ISBN'] == most_common_other_book) |
    (books_of_users_with_high_book['ISBN'] == most_highly_rated_book)
]['User-ID'].value_counts()
users_with_both_books_count = sum(users_with_both_books == 2)

print("Users with Both Books:", users_with_both_books_count)


percentage_users_with_both = (users_with_both_books_count / len(users_with_most_highly_rated_book)) * 100

print(f"Percentage of users who have both the most highly rated book and the most common other book: {percentage_users_with_both:.2f}%")

User-book Ratings Merged Overview:
    User-ID        ISBN  Book-Rating  \
1    276726  0155061224            5   
3    276729  052165615X            3   
4    276729  0521795028            6   
6    276744  038550120X            7   
13   276747  0060517794            9   

                                           Book-Title    Book-Author  \
1                                    Rites of Passage     Judith Rae   
3                                      Help!: Level 1  Philip Prowse   
4   The Amsterdam Connection : Level 4 (Cambridge ...    Sue Leather   
6                                     A Painted House   JOHN GRISHAM   
13                           Little Altars Everywhere  Rebecca Wells   

   Year-Of-Publication                   Publisher  \
1                 2001                      Heinle   
3                 1999  Cambridge University Press   
4                 2001  Cambridge University Press   
6                 2001                   Doubleday   
13                200